In [1]:
import torch
import gpytorch
import time

%pdb

Automatic pdb calling has been turned ON


In [2]:
n = 1000

train_x = torch.randn(n, 1)
kernel = gpytorch.kernels.keops.SpectralGPKernel(train_x, register_latent_params=True)
rhs = torch.randn(n, 1)

In [3]:
list(kernel.named_parameters()) # latent params requires gradients?

[('latent_params', Parameter containing:
  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.], requires_grad=True))]

In [4]:
res = kernel(train_x).evaluate_kernel().matmul(torch.randn(1000))
res.requires_grad

does density require grad inside of the covar func True
inside of the covar func, does an evaluation require grad:  True
does density require grad inside of the covar func True
inside of the covar func, does an evaluation require grad:  True
here the results rq? True
in the Matmul forward Function, does the result require grad True


RuntimeError: Returning Variables sharing storage with other Variables that require grad is not supported in Python functions. Please submit a feature request if you hit this error.

> /home/wesley/Documents/Code/wjm_gpytorch/gpytorch/lazy/lazy_tensor.py(1102)matmul()
   1100 
   1101         func = Matmul()
-> 1102         result = func.apply(self.representation_tree(), other, *self.representation())
   1103         print('in lazy tensor matmul, does the result require grad', result.requires_grad)
   1104         return result

ipdb> q


In [5]:
# googling results in this error
# https://github.com/pytorch/pytorch/issues/32625
# can't tell if this is my error or what i'm doing wrong in pytorch any more